In [1]:
import sys
!{sys.executable} -m pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 16.9 MB/s eta 0:00:00


In [2]:
import sys
!{sys.executable} -m pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 5.1 MB/s eta 0:00:00


In [3]:
#import sys
#!{sys.executable} -m pip install scikeras

In [4]:
import sys
!{sys.executable} -m pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=32c8aa7875b2682e8723d0ce22c2c83a1e3f63c6a7f5af44d4382d4ad2db46bc
  Stored in directory: /home/idies/.cache/pip/wheels/20/7c/09/4ad42725a29fce4bc21137c7f25f062b3655a4aea5b0e8d9a2
Successfully built pathtools
  Attempting uninstall: GitPython
    Found existing installation: GitPython 3.1.29
    Uninstalling GitPython-3.1.29:
      Successfully uninstalled GitPython-3.1.29


In [5]:
import os
os.environ['WANDB_API_KEY'] = 'b3c6dfb61d66930ee7d291c6b19c0ae0b5007c62'

In [6]:
import wandb
wandb.init(project="exoplanetml", entity="sakhawat",save_code=True,name='NeuralNet RandomSearch',job_type='analysis',reinit=True)

wandb: Currently logged in as: sakhawat. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /home/idies/workspace/Temporary/s.hossain18/scratch/jobs/20230821/20230821022955-245779/wandb/run-20230821_023104-y5rdxauh
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run NeuralNet RandomSearch
wandb: ⭐️ View project at https://wandb.ai/sakhawat/exoplanetml
wandb: 🚀 View run at https://wandb.ai/sakhawat/exoplanetml/runs/y5rdxauh


In [7]:
import numpy as np

In [8]:
X_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy')
X_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test_pca.npy')
y_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy')
y_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy')

In [9]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from scikeras.wrappers import KerasClassifier

In [11]:
# Define a function to create the neural network model
def create_model(hidden_layer_sizes, activation, solver, alpha):
    model = Sequential()
    model.add(Dense(64, activation='relu'))
    for size in hidden_layer_sizes:
        model.add(Dense(size, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=solver, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [12]:
# Create an instance of the KerasClassifier using the defined model function
keras_classifier = KerasClassifier(build_fn=create_model,verbose=0)

/tmp/ipykernel_11/702602205.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_classifier = KerasClassifier(build_fn=create_model,verbose=0)


In [13]:
#from skopt.space import Real, Integer, Categorical
# ...
# ...
params = {
    'hidden_layer_sizes': [(100,), (150,), (200,), (250,)],
    'activation': ['sigmoid', 'softplus', 'tanh', 'relu'],
    'solver': ['adamax', 'sgd', 'adam'],
    'alpha': np.logspace(-5, -2, base=10)
}
# ...

In [14]:
# Log the hyperparameters to wandb.config
wandb.config.update(params)

In [15]:
# Create an instance of BayesianSearchCV
random_cv = RandomizedSearchCV(keras_classifier, params, n_iter=100, cv=cv, n_jobs=-1)

In [16]:
# Perform hyperparameter tuning using Bayesian optimization
random_cv.fit(X_train, np.ravel(y_train))

/home/idies/mambaforge/envs/py39/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=10, random_state=41),
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f39b7ac0c70>,
                   n_iter=100, n_jobs=-1,
                   param_distributions={'activation': ['sigmoid', 'softplus',
                                                       'tanh', 'relu'],
                                        'alpha': array([1.00000000e-05, 1.15139540e-05, 1.32571137e-05, 1.52641797e-05,
       1.75751062e-05, 2....
       9.10298178e-04, 1.04811313e-03, 1.20679264e-03, 1.38949549e-03,
       1.59985872e-03, 1.84206997e-03, 2.12095089e-03, 2.44205309e-03,
       2.81176870e-03, 3.23745754e-03, 3.72759372e-03, 4.29193426e-03,
       4.94171336e-03, 5.68986603e-03, 6.55128557e-03, 7.54312006e-03,
       8.68511374e-03, 1.00000000e-02]),
                                        'hidden_layer_sizes': [(100,), (150,),
                                                               (200,), (250,)],
                                        'solver': ['adamax', 'sgd', 'adam']})

In [17]:
# Print the best score and hyperparameters
print("Train score:", random_cv.best_score_)
print("Test score:", random_cv.score(X_test, y_test))
print("Best hyperparameters:", random_cv.best_params_)

Train score: 0.044014779329299925
Test score: 0.0
Best hyperparameters: {'solver': 'sgd', 'hidden_layer_sizes': (200,), 'alpha': 0.0005179474679231213, 'activation': 'tanh'}


In [18]:
# Log the best score and hyperparameters
wandb.log({"Train score": random_cv.best_score_})
wandb.log({"Test score": random_cv.score(X_test, y_test)})
wandb.log({"Best hyperparameters": random_cv.best_params_})

In [19]:
# Finish logging with wandb
wandb.finish()

wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:  Test score ▁
wandb: Train score ▁
wandb: 
wandb: Run summary:
wandb:  Test score 0.0
wandb: Train score 0.04401
wandb: 
wandb: 🚀 View run NeuralNet RandomSearch at: https://wandb.ai/sakhawat/exoplanetml/runs/y5rdxauh
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230821_023104-y5rdxauh/logs


In [20]:
#keras_classifier.get_params().keys()